# Optimal triangulation of implicity surfaces

In [ ]:
from sympy.abc import x,y,z
from sympy import lambdify,diff, sqrt, simplify
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from scipy.optimize import minimize
from numba import jit

def compute_jet(ff, xs, ys, zs):
    f = lambdify([x,y,z],ff)
    fx = lambdify([x,y,z], diff(ff,x))
    fy = lambdify([x,y,z], diff(ff,y))
    fz = lambdify([x,y,z], diff(ff,z))
    fxs = fx(xs,ys,zs)
    fys = fy(xs,ys,zs)
    fzs = fz(xs,ys,zs)
    ns = np.sqrt(fxs**2 + fys**2 + fzs**2)
    return f(xs,ys,zs), fxs/ns, fys/ns, fzs/ns

def grid_xyz(box, res):
    xmin, xmax, ymin, ymax, zmin, zmax = box
    xs = np.linspace(xmin, xmax, res)
    ys = np.linspace(ymin, ymax, res)
    zs = np.linspace(zmin, zmax, res)
    X,Y,Z = np.meshgrid(xs, ys, zs)
    X = X.flatten()
    Y = Y.flatten()
    Z = Z.flatten()
    return X, Y, Z

res = 2
# Load the data
base = "../SphereTriangulation/"
sph_fs = np.loadtxt(base+'triangles%s.txt'%res)
sph_vs = np.loadtxt(base+'vertices%s.txt'%res)

print("Spherical points count: %s"%sph_vs.shape[0])

# plot shperical triangles
fig = go.Figure()

# plot sph_vs
fig.add_trace(go.Scatter3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], mode='markers', marker=dict(size=2, color='red')))
# set width and height
fig.update_layout(width=600, height=600)

fig.show()

In [ ]:
R = 2
r = 1
ax = .1
ay = .1
ff = (sqrt(x**2+y**2)- R)**2 + (z-ax*x**2+ay*y**2)**2 - r**2
box = [-3,3,-3,3,-1.5,1.5]
X, Y, Z = grid_xyz(box,100)
f, nx, ny, nz = compute_jet(ff, X, Y, Z)

dir = np.array([1,1,0])
dir = dir/np.linalg.norm(dir)
print("dir: %s"%dir)

cf = np.abs(f) < 0.02
cn = np.sqrt((nx-dir[0])**2 + (ny-dir[1])**2 + (nz-dir[2])**2) < 0.05
w1, = np.where(cf)
w2, = np.where(cf & cn)

fig = go.Figure()

# plot sph_vs
fig.add_trace(go.Scatter3d(x=X[w1], y=Y[w1], z=Z[w1], mode='markers', marker=dict(size=1, color='red')))
fig.add_trace(go.Scatter3d(x=X[w2], y=Y[w2], z=Z[w2], mode='markers', marker=dict(size=2, color='blue')))

# set width and height
fig.update_layout(width=800, height=800)
fig.show()

In [ ]:

def energy_function(f, fx, fy, fz, n):
    def energy(p):
        e1 = f(*p)**2
        n0 = np.sqrt(n[0]**2 + n[1]**2 + n[2]**2)
        fx0 = n[0]/n0
        fy0 = n[1]/n0
        fz0 = n[2]/n0
        fxv = fx(*p)
        fyv = fy(*p)
        fzv = fz(*p)
        nrm = np.sqrt(fxv**2 + fyv**2 + fzv**2)
        fxv = fxv/nrm
        fyv = fyv/nrm
        fzv = fzv/nrm
        e2 = (fx(*p)-fx0)**2 + (fy(*p)-fy0)**2 + (fz(*p)-fz0)**2
        return 10*e1 + e2
    return energy

def inverse_gauss_map(ff, ns, points_count=100):
    f = lambdify([x,y,z],ff, 'numpy')
    fx = lambdify([x,y,z], diff(ff,x), 'numpy')
    fy = lambdify([x,y,z], diff(ff,y), 'numpy')
    fz = lambdify([x,y,z], diff(ff,z), 'numpy')
    
    all_ps = []
    for n in ns:
        energy = energy_function(f, fx, fy, fz, n)
        # choose random points in box
        xs = np.random.uniform(box[0], box[1], points_count)
        ys = np.random.uniform(box[2], box[3], points_count)
        zs = np.random.uniform(box[4], box[5], points_count)
        ps = []
        for i in range(points_count):
            res = minimize(energy, [xs[i],ys[i],zs[i]])
            ps.append(res.x)
        ps = np.array(ps)
        # Clustering points
        clustering = DBSCAN(eps=0.1, min_samples=1).fit(ps)
        lbs = clustering.labels_
        count = len(np.unique(lbs))
        ps = np.array([np.mean(ps[np.where(lbs==i)[0]],axis=0) for i in range(count)])
        all_ps.append(ps)
    return all_ps

ns = np.array([[0,0,-1]])
#ns = sph_vs
ips = inverse_gauss_map(ff, ns, 20)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=X[w1], y=Y[w1], z=Z[w1], mode='markers', marker=dict(size=1, color='red')))

for ps in ips:
    fig.add_trace(go.Scatter3d(x=ps[:,0], y=ps[:,1], z=ps[:,2], mode='markers', marker=dict(size=4, color='blue')))

#fig.add_trace(go.Scatter3d(x=ips[:,0], y=ips[:,1], z=ips[:,2], mode='markers', marker=dict(size=4, color='blue')))

fig.update_layout(width=600, height=600)
fig.show()